In [34]:
import pandas as pd
import numpy as np
import csv

In [35]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("trg.csv")
df = df.rename({'class':'target'},axis=1)
training, testing = train_test_split(df, test_size=0.2, random_state=42)



In [36]:
vocab = {word for para in training['abstract'] for word in para.split()}
print(len(vocab))
prior_probability = training['target'].value_counts(normalize=True).to_dict()
prior_probability 

28371


{'E': 0.5434375, 'B': 0.3915625, 'A': 0.033125, 'V': 0.031875}

In [37]:
word_orrurance = dict()
for row in training.itertuples(index = 0): 
    line = row.abstract
    target = row.target
    # print(target)
    for word in line.split():
         
        if word in word_orrurance.keys():
            if target in word_orrurance[word].keys():
                word_orrurance[word][target] += 1
            else:
                word_orrurance[word][target] = 1
        else:
            word_orrurance[word] = {target:1}

# print(word_orrurance)

data = []
for word, targets in word_orrurance.items():
    
    row = {'word': word}
    
    for i, (target, count) in enumerate(targets.items()):
        row [target] = count
    # append the row to the data list
    data.append(row)
word_orrurance_table = pd.DataFrame(data)
word_orrurance_table.fillna(0, inplace=True)
word_orrurance_table
print(word_orrurance_table.loc[word_orrurance_table['word'] == 'virus'])
word_orrurance_table

       word    B     E    A      V
2641  virus  8.0  38.0  0.0  207.0


,word,B,E,A,V
0,shewanella,13.0,0.0,0.0,0.0
1,oneidensis,18.0,0.0,0.0,0.0
2,is,2406.0,3344.0,162.0,158.0
3,an,917.0,1275.0,154.0,73.0
4,important,213.0,192.0,8.0,12.0
...,...,...,...,...,...
28366,bpha,0.0,1.0,0.0,0.0
28367,benzoate-p-hydroxylase,0.0,1.0,0.0,0.0
28368,sugar-beet,0.0,2.0,0.0,0.0
28369,receptively,0.0,1.0,0.0,0.0


In [38]:
total_B = word_orrurance_table['B'].sum()
total_E = word_orrurance_table['E'].sum()
total_A = word_orrurance_table['A'].sum()
total_V = word_orrurance_table['V'].sum()
# print("B",total_B)  
# print("E",total_E)
# print("A",total_A)
# print("V", total_V)

In [39]:
conditional_probability = word_orrurance_table.copy()

conditional_probability['B'] = conditional_probability['B'] / (total_B)
conditional_probability['E'] = conditional_probability['E'] / (total_E)
conditional_probability['A'] = conditional_probability['A'] / (total_A)
conditional_probability['V'] = conditional_probability['V'] / (total_V)

conditional_probability

,word,B,E,A,V
0,shewanella,0.000058,0.000000,0.000000,0.000000
1,oneidensis,0.000080,0.000000,0.000000,0.000000
2,is,0.010727,0.010900,0.007063,0.008346
3,an,0.004088,0.004156,0.006714,0.003856
4,important,0.000950,0.000626,0.000349,0.000634
...,...,...,...,...,...
28366,bpha,0.000000,0.000003,0.000000,0.000000
28367,benzoate-p-hydroxylase,0.000000,0.000003,0.000000,0.000000
28368,sugar-beet,0.000000,0.000007,0.000000,0.000000
28369,receptively,0.000000,0.000003,0.000000,0.000000
